In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import precision_recall_fscore_support

# read in flight data
flight = pd.read_csv('data/raw/Airline_Delay_Cause.csv')
late_threshhold = 20 * 60
flight['late'] = np.where(flight['late_aircraft_delay'] <= late_threshhold, 0, 1) # collapse values in column

# standardize data
flight['arr_flights'] = (flight.arr_flights - flight.arr_flights.mean()) / flight.arr_flights.std()
flight['arr_del15'] = (flight.arr_del15 - flight.arr_del15.mean()) / flight.arr_del15.std()
flight['carrier_ct'] = (flight.carrier_ct - flight.carrier_ct.mean()) / flight.carrier_ct.std()
flight['weather_ct'] = (flight.weather_ct - flight.weather_ct.mean()) / flight.weather_ct.std()
flight['nas_ct'] = (flight.nas_ct - flight.nas_ct.mean()) / flight.nas_ct.std()
flight['security_ct'] = (flight.security_ct - flight.security_ct.mean()) / flight.security_ct.std()
flight['late_aircraft_ct'] = (flight.late_aircraft_ct - flight.late_aircraft_ct.mean()) / flight.late_aircraft_ct.std()
flight['arr_cancelled'] = (flight.arr_cancelled - flight.arr_cancelled.mean()) / flight.arr_cancelled.std()
flight['arr_diverted'] = (flight.arr_diverted - flight.arr_diverted.mean()) / flight.arr_diverted.std()
flight['arr_delay'] = (flight.arr_delay - flight.arr_delay.mean()) / flight.arr_delay.std()
flight['carrier_delay'] = (flight.carrier_delay - flight.carrier_delay.mean()) / flight.carrier_delay.std()
flight['weather_delay'] = (flight.weather_delay - flight.weather_delay.mean()) / flight.weather_delay.std()
flight['nas_delay'] = (flight.nas_delay - flight.nas_delay.mean()) / flight.nas_delay.std()
flight['security_delay'] = (flight.security_delay - flight.security_delay.mean()) / flight.security_delay.std()
flight['late_aircraft_delay'] = (flight.late_aircraft_delay - flight.late_aircraft_delay.mean()) / flight.late_aircraft_delay.std()

flight = flight.dropna()

# perform knn
flightsX = flight[['arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct', 'nas_ct', 'security_ct', 'late_aircraft_ct', 'arr_cancelled', 
                   'arr_diverted', 'arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']].values
flightsy = flight[['late']].values
X_train, X_test, y_train, y_test = train_test_split(flightsX, flightsy, test_size = 0.4)
nn = NearestNeighbors(n_neighbors=5, metric='euclidean', algorithm='auto')
fit = nn.fit(flightsX)
distances, indices = fit.kneighbors(flightsX)

NameError: name 'flightsx' is not defined

In [ ]:
display(len(flightsX))

y_pred = []
for i in range(len(flightsX)):
    nbrs = flight.iloc[indices[i]]
    nbrs = nbrs.drop(flight.index[i], errors='ignore')
    ontime = nbrs[nbrs.late == 0].count().late
    late = nbrs[nbrs.late == 1].count().late
    predict = 0 if (ontime > late) else 1
    y_pred.append(predict)   
    display(i)


99931

0

1

2

3

...

79593

79594

79595

79596

79597

In [12]:
(p, r, f, s) = precision_recall_fscore_support(flightsy, y_pred, labels=[1])
display(p)
display(r)
display(f)

array([0.92161814])

array([0.95379457])

array([0.93743033])